# About: O07_02_Hadoop_Restore a "Slave Node"

---

In this notebook, **restore a Slave Node.**



## *Operation Note*

*This is a cell for your own recording.  ここに経緯を記述*

# Confirm Inventory

Review the structure definition - `hosts.csv`:

In [1]:
target_group = 'hadoop_all_cluster1'

In [2]:
target_hostname = 'sn02031601'

In [3]:
%run scripts/loader.py

TARGET_CLUSTER = 'Cluster1'

header, machines = read_machines("hosts.csv")
machines = [m for m in machines if m['Cluster'] == TARGET_CLUSTER]

print("Cluster(%s):" % TARGET_CLUSTER)
pd.DataFrame([get_row(header, m) for m in machines], columns=header)

Cluster(Cluster1):


,Cluster,Type,Name,Internal IP,Service IP,VCPUs,Memory(MiB),DFS Volumes,YARN VCPUs,YARN Total Memory(MB),...,HBase RegionServer,Tez,Hive,Pig,Client,Spark,Spark HistoryServer,KDC Master,KDC Slave,Docker
0,Cluster1,cn0107,cn01070401,XXX.XXX.XXX.85,XXX.XXX.XXX.197,12,98304,NaN,NaN,NaN,...,False,False,False,False,False,False,False,True,False,False
1,Cluster1,cn0107,cn01070402,XXX.XXX.XXX.86,XXX.XXX.XXX.198,12,98304,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,True,False
2,Cluster1,cn0107,cn01070403,XXX.XXX.XXX.87,XXX.XXX.XXX.199,12,98304,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,Cluster1,cn0107,cn01070404,XXX.XXX.XXX.88,XXX.XXX.XXX.200,12,98304,NaN,NaN,NaN,...,False,True,True,True,True,True,True,False,False,False
4,Cluster1,cn0107,cn01070603,XXX.XXX.XXX.83,XXX.XXX.XXX.195,12,98304,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
5,Cluster1,cn0107,cn01070604,XXX.XXX.XXX.84,XXX.XXX.XXX.196,12,98304,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,True
6,Cluster1,sn0202,sn02020401,XXX.XXX.XXX.12,XXX.XXX.XXX.230,16,65536,10.0,15.0,64512.0,...,True,False,False,False,False,False,False,False,False,False
7,Cluster1,sn0202,sn02021201,XXX.XXX.XXX.8,XXX.XXX.XXX.228,16,65536,10.0,15.0,64512.0,...,True,False,False,False,False,False,False,False,False,False
8,Cluster1,sn0202,sn02022001,XXX.XXX.XXX.4,XXX.XXX.XXX.226,16,65536,10.0,15.0,64512.0,...,True,False,False,False,False,False,False,False,False,False
9,Cluster1,sn0202,sn02022401,XXX.XXX.XXX.2,XXX.XXX.XXX.225,16,65536,10.0,15.0,64512.0,...,True,False,False,False,False,False,False,False,False,False


In [4]:
target_hosts = filter(lambda m: m['Name'] == target_hostname, machines)
assert(target_hosts)

---
For following sections remember new server's Service NIC address

In [5]:
host_new_machine = target_hosts[0]['Service IP']
host_new_machine

'XXX.XXX.XXX.233'

# Initialize and mount new HDD

## Retrieve utility playbooks

We would like to use utility playbooks to manage volumes. The playbooks are **PRIVATE** because they include our private information...

In [6]:
import tempfile
prereq_path = tempfile.mkdtemp()
!mkdir -p {prereq_path}/original
!git clone ssh://xxx.nii.ac.jp/xxx/aic-dataanalysis-prerequisite.git {prereq_path}/original
!ls -la {prereq_path}

Cloning into '/tmp/tmpwDTh_U/original'...
remote: Counting objects: 399, done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 399 (delta 138), reused 196 (delta 55)
Receiving objects: 100% (399/399), 37.95 KiB | 0 bytes/s, done.
Resolving deltas: 100% (138/138), done.
Checking connectivity... done.
total 12
drwx------  3 root root 4096 Sep  2 18:56 .
drwxrwxrwt 65 root root 4096 Sep  2 18:56 ..
drwxr-xr-x  7 root root 4096 Sep  2 18:56 original


In [7]:
!mkdir -p {prereq_path}/current
%run common/inventory-base.py
import os

with open(os.path.join(prereq_path, 'current', 'hosts'), 'w') as f:
    write_base_inventory(target_hosts, f)
!cat {prereq_path}/current/hosts

[sn02031601]
XXX.XXX.XXX.233

[sn0203:children]
sn02031601

[Cluster1:children]
sn02031601

[ganglia_masters:children]

[kerberos_servers:children]
kdc_masters
kdc_slaves

[kdc_masters:children]

[kdc_masters:vars]
kdc_role=master

[kdc_slaves:children]

[kdc_slaves:vars]
kdc_role=slave

[kerberos_clients:children]



In [8]:
import yaml
with open(os.path.join(prereq_path, 'original', 'group_vars', target_hosts[0]['Type']), 'r') as f:
    group_vars = yaml.load(f)
group_vars

{'bonding_nic_1': '',
 'bonding_nic_2': '',
 'hdd_devices_and_mountpoints': [{'device': '/dev/sdc',
   'mount': '/hadoop/tmp'},
  {'device': '/dev/sdd', 'mount': '/hadoop/data01'},
  {'device': '/dev/sde', 'mount': '/hadoop/data02'},
  {'device': '/dev/sdf', 'mount': '/hadoop/data03'},
  {'device': '/dev/sdg', 'mount': '/hadoop/data04'},
  {'device': '/dev/sdh', 'mount': '/hadoop/data05'},
  {'device': '/dev/sdi', 'mount': '/hadoop/data06'},
  {'device': '/dev/sdj', 'mount': '/hadoop/data07'},
  {'device': '/dev/sdk', 'mount': '/hadoop/data08'},
  {'device': '/dev/sdl', 'mount': '/hadoop/data09'},
  {'device': '/dev/sdm', 'mount': '/hadoop/data10'}],
 'log_device': '/dev/sdb',
 'server_nic_type': 'sn0203',
 'server_type': 'sn'}

Test the inventory...

In [9]:
!ansible -m ping -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## Add Logical Disk to use the HDD

In [10]:
!ansible -b -a '/opt/MegaRAID/MegaCli/MegaCli64 -PDList -a0' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
                                     
Adapter #0

Enclosure Device ID: 32
Slot Number: 0
Drive's position: DiskGroup: 1, Span: 0, Arm: 0
Enclosure position: 1
Device Id: 0
WWN: 50014EE55556CA41
Sequence Number: 2
Media Error Count: 0
Other Error Count: 0
Predictive Failure Count: 0
Last Predictive Failure Event Seq Number: 0
PD Type: SAS

Raw Size: 2.728 TB [0x15d50a3b0 Sectors]
Non Coerced Size: 2.728 TB [0x15d40a3b0 Sectors]
Coerced Size: 2.728 TB [0x15d400000 Sectors]
Sector Size:  0
Firmware state: Online, Spun Up
Device Firmware Level: D1R2
Shield Counter: 0
Successful diagnostics completion on :  N/A
SAS Address(0): 0x50014ee55556ca42
SAS Address(1): 0x0
Connected Port Number: 0(path0) 
Inquiry Data: WD      WD3001FYYG      D1R2WCC1F0082655        
FDE Capable: Not Capable
FDE Enable: Disable
Secured: Unsecured
Locked: Unlocked
Needs EKM Attention: No
Foreign State: None 
Device Speed: 6.0Gb/s 
Link Speed: 6.0Gb/s 
Media Type: Hard Disk Device


The `Enclosure Device ID:Slot Number` for the new disk is... **32:6**

In [11]:
new_hdd_location = (32, 6)

Checking all Logical Drives...

In [12]:
!ansible -b -a '/opt/MegaRAID/MegaCli/MegaCli64 -LDInfo -Lall -a0' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
                                     

Adapter 0 -- Virtual Drive Information:
Virtual Drive: 0 (Target Id: 0)
Name                :
RAID Level          : Primary-1, Secondary-0, RAID Level Qualifier-0
Size                : 136.125 GB
Sector Size         : 512
Mirror Data         : 136.125 GB
State               : Optimal
Strip Size          : 64 KB
Number Of Drives    : 2
Span Depth          : 1
Default Cache Policy: WriteBack, ReadAdaptive, Direct, No Write Cache if Bad BBU
Current Cache Policy: WriteThrough, ReadAheadNone, Direct, No Write Cache if Bad BBU
Default Access Policy: Read/Write
Current Access Policy: Read/Write
Disk Cache Policy   : Disk's Default
Encryption Type     : None
Default Power Savings Policy: Controller Defined
Current Power Savings Policy: None
Can spin up in 1 minute: Yes
LD has drives that support T10 power conditions: Yes
LD's IO profile supports MAX power savings with cached writes: No
Bad Blocks Exist: No
Is VD Cached:

Confirmed that **Logical Device 7** is missing.

Add the new disk as a logical disk

In [13]:
!ansible -b -a '/opt/MegaRAID/MegaCli/MegaCli64 -CfgLdAdd -r0 [{new_hdd_location[0]}:{new_hdd_location[1]}] -a0' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | FAILED | rc=84 >>
                                     

Adapter 0: Configure Adapter Failed

FW error description: 
  The current operation is not allowed because the controller has data in cache for offline or missing virtual drives.  

Exit Code: 0x54



Oops, the preserved cache remains... OK, let's remove it.

Set the old LD.

In [14]:
previous_ld_number = 7

In [15]:
!ansible -b -a '/opt/MegaRAID/MegaCli/MegaCli64 -DiscardPreservedCache -L{previous_ld_number} -a0' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
                                     
Adapter #0

Virtual Drive(Target ID 07): Preserved Cache Data Cleared.

Exit Code: 0x00



OK, try to create LD...

In [16]:
!ansible -b -a '/opt/MegaRAID/MegaCli/MegaCli64 -CfgLdAdd -r0 [{new_hdd_location[0]}:{new_hdd_location[1]}] -a0' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
                                     
Adapter 0: Created VD 7

Adapter 0: Configured the Adapter!!

Exit Code: 0x00



np...! The number of new VD is **7**

Then, I'm going to **reboot the machine and recreate all volumes**.
Rainy, let's dry run...

In [17]:
!ansible -CDv -b -a 'reboot' -i {prereq_path}/current/hosts all

Using /etc/ansible/ansible.cfg as config file
XXX.XXX.XXX.233 | SKIPPED


OK, reboot it...!

In [18]:
!ansible -b -a 'reboot' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>




Wait for the machine...

In [32]:
!ping -c 4 {host_new_machine}

PING XXX.XXX.XXX.233 (XXX.XXX.XXX.233) 56(84) bytes of data.
64 bytes from XXX.XXX.XXX.233: icmp_seq=1 ttl=63 time=0.365 ms
64 bytes from XXX.XXX.XXX.233: icmp_seq=2 ttl=63 time=0.323 ms
64 bytes from XXX.XXX.XXX.233: icmp_seq=3 ttl=63 time=0.290 ms
64 bytes from XXX.XXX.XXX.233: icmp_seq=4 ttl=63 time=0.350 ms

--- XXX.XXX.XXX.233 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 2997ms
rtt min/avg/max/mdev = 0.290/0.332/0.365/0.028 ms


In [33]:
!ansible -m ping {host_new_machine}

XXX.XXX.XXX.233 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


Review mountpoints...

In [34]:
!ansible -a 'df -H' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda2        14G  4.0G  9.3G  30% /
tmpfs            34G     0   34G   0% /dev/shm
/dev/sda5       726M  180M  509M  27% /mnt
/dev/sdb1       3.0T  7.1G  2.8T   1% /var/log
/dev/sdc1       3.0T   19G  2.8T   1% /hadoop/tmp
/dev/sdd1       3.0T   46G  2.8T   2% /hadoop/data01
/dev/sde1       3.0T   51G  2.8T   2% /hadoop/data02
/dev/sdk1       3.0T   34G  2.8T   2% /hadoop/data03
/dev/sdg1       3.0T   48G  2.8T   2% /hadoop/data04
/dev/sdi1       3.0T   48G  2.8T   2% /hadoop/data06
/dev/sdj1       3.0T   48G  2.8T   2% /hadoop/data07
/dev/sdl1       3.0T   50G  2.8T   2% /hadoop/data09
/dev/sdm1       3.0T   50G  2.8T   2% /hadoop/data10



In [35]:
import yaml
with open(os.path.join(prereq_path, 'original', 'group_vars', target_hosts[0]['Type']), 'r') as f:
    group_vars = yaml.load(f)
group_vars

{'bonding_nic_1': '',
 'bonding_nic_2': '',
 'hdd_devices_and_mountpoints': [{'device': '/dev/sdc',
   'mount': '/hadoop/tmp'},
  {'device': '/dev/sdd', 'mount': '/hadoop/data01'},
  {'device': '/dev/sde', 'mount': '/hadoop/data02'},
  {'device': '/dev/sdf', 'mount': '/hadoop/data03'},
  {'device': '/dev/sdg', 'mount': '/hadoop/data04'},
  {'device': '/dev/sdh', 'mount': '/hadoop/data05'},
  {'device': '/dev/sdi', 'mount': '/hadoop/data06'},
  {'device': '/dev/sdj', 'mount': '/hadoop/data07'},
  {'device': '/dev/sdk', 'mount': '/hadoop/data08'},
  {'device': '/dev/sdl', 'mount': '/hadoop/data09'},
  {'device': '/dev/sdm', 'mount': '/hadoop/data10'}],
 'log_device': '/dev/sdb',
 'server_nic_type': 'sn0203',
 'server_type': 'sn'}

In [36]:
!mkdir -p {prereq_path}/current/group_vars
del group_vars['log_device']
with open(os.path.join(prereq_path, 'current', 'group_vars', target_hosts[0]['Type']), 'w') as f:
    f.write(yaml.dump(group_vars))
!cat {prereq_path}/current/group_vars/{target_hosts[0]['Type']}

bonding_nic_1: ''
bonding_nic_2: ''
hdd_devices_and_mountpoints:
- {device: /dev/sdc, mount: /hadoop/tmp}
- {device: /dev/sdd, mount: /hadoop/data01}
- {device: /dev/sde, mount: /hadoop/data02}
- {device: /dev/sdf, mount: /hadoop/data03}
- {device: /dev/sdg, mount: /hadoop/data04}
- {device: /dev/sdh, mount: /hadoop/data05}
- {device: /dev/sdi, mount: /hadoop/data06}
- {device: /dev/sdj, mount: /hadoop/data07}
- {device: /dev/sdk, mount: /hadoop/data08}
- {device: /dev/sdl, mount: /hadoop/data09}
- {device: /dev/sdm, mount: /hadoop/data10}
server_nic_type: sn0203
server_type: sn


Remove all files on all volumes...

In [37]:
for hdd in group_vars['hdd_devices_and_mountpoints']:
    if hdd['mount'] == '/hadoop/tmp':
        dirs = os.path.join(hdd['mount'], 'hadoop-yarn')
    else:
        dirs = os.path.join(hdd['mount'], 'dfs')
    !ansible -b -a 'rm -fr {dirs}' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>


XXX.XXX.XXX.233 | SUCCESS | rc=0 >>




## Format the HDD

In [38]:
!cp {prereq_path}/original/playbooks/destructive/clean-hdds.yml {prereq_path}/current/

First, let's dry run...!

In [39]:
!ansible-playbook -CDv -i {prereq_path}/current/hosts {prereq_path}/current/clean-hdds.yml

Using /etc/ansible/ansible.cfg as config file
[DEPRECATION WARNING]: Instead of sudo/sudo_user, use become/become_user and 
make sure become_method is 'sudo' (default).
This feature will be removed in a 
future release. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.

PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [install gdisk package.] **************************************************
ok: [XXX.XXX.XXX.233] => {"changed": false, "msg": "", "rc": 0, "results": ["gdisk-0.8.10-1.el6.x86_64 providing gdisk is already installed"]}

TASK [unmount disks to be formatted.] ******************************************
skipping: [XXX.XXX.XXX.233] => {"changed": false, "skip_reason": "Conditional check failed", "skipped": true}

TASK [zap GPT and MBR data structure of HDD.] **********************************
ski

In [40]:
!ansible-playbook -i {prereq_path}/current/hosts {prereq_path}/current/clean-hdds.yml

[DEPRECATION WARNING]: Instead of sudo/sudo_user, use become/become_user and 
make sure become_method is 'sudo' (default).
This feature will be removed in a 
future release. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.

PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [install gdisk package.] **************************************************
ok: [XXX.XXX.XXX.233]

TASK [unmount disks to be formatted.] ******************************************
skipping: [XXX.XXX.XXX.233]

TASK [zap GPT and MBR data structure of HDD.] **********************************
skipping: [XXX.XXX.XXX.233]

TASK [create partition.] *******************************************************
skipping: [XXX.XXX.XXX.233]

TASK [unmount disks to be formatted.] ******************************************
[DEPRECATION WARNING]: Using bar

## Mount the disk

In [41]:
!cp -fr {prereq_path}/original/roles {prereq_path}/current/roles

In [42]:
%%writefile {prereq_path}/current/mount.yml
- hosts: all
  become: yes
  roles:
    - mount

Writing /tmp/tmpwDTh_U/current/mount.yml


In [43]:
!ansible-playbook -CDv -i {prereq_path}/current/hosts {prereq_path}/current/mount.yml

Using /etc/ansible/ansible.cfg as config file

PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [mount : mkfs for /var/log] ***********************************************
skipping: [XXX.XXX.XXX.233] => {"changed": false, "skip_reason": "Conditional check failed", "skipped": true}

TASK [mount : create a temporary directory for previous /var/log] **************
skipping: [XXX.XXX.XXX.233] => {"changed": false, "skip_reason": "Conditional check failed", "skipped": true}

TASK [mount : copy files in /var/log to /tmp/old-log to keep logs] *************
skipping: [XXX.XXX.XXX.233] => {"changed": false, "skip_reason": "Conditional check failed", "skipped": true}

TASK [mount : mount /var/log] **************************************************
skipping: [XXX.XXX.XXX.233] => {"changed": false, "skip_reason": "Conditional check failed", "skipped": true}


In [44]:
!ansible-playbook -i {prereq_path}/current/hosts {prereq_path}/current/mount.yml


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [mount : mkfs for /var/log] ***********************************************
skipping: [XXX.XXX.XXX.233]

TASK [mount : create a temporary directory for previous /var/log] **************
skipping: [XXX.XXX.XXX.233]

TASK [mount : copy files in /var/log to /tmp/old-log to keep logs] *************
skipping: [XXX.XXX.XXX.233]

TASK [mount : mount /var/log] **************************************************
skipping: [XXX.XXX.XXX.233]

TASK [mount : make filesystem on devices] **************************************
[DEPRECATION WARNING]: Using bare variables is deprecated. Update your playbooks
 so that the environment value uses the full variable syntax 
('{{hdd_devices_and_mountpoints}}').
This feature will be removed in a future 
release. Deprecation warnings can be disabled by setting 
deprecation_

Review the mountpoint...

In [45]:
!ansible -a 'df -H' -i {prereq_path}/current/hosts all

XXX.XXX.XXX.233 | SUCCESS | rc=0 >>
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda2        14G  4.0G  9.3G  30% /
tmpfs            34G     0   34G   0% /dev/shm
/dev/sda5       726M  180M  509M  27% /mnt
/dev/sdb1       3.0T  7.1G  2.8T   1% /var/log
/dev/sdc1       3.0T  7.9G  2.8T   1% /hadoop/tmp
/dev/sdd1       3.0T   77M  2.9T   1% /hadoop/data01
/dev/sde1       3.0T   77M  2.9T   1% /hadoop/data02
/dev/sdf1       3.0T   22G  2.8T   1% /hadoop/data03
/dev/sdg1       3.0T   77M  2.9T   1% /hadoop/data04
/dev/sdh1       4.0T   72M  3.8T   1% /hadoop/data05
/dev/sdi1       3.0T   77M  2.9T   1% /hadoop/data06
/dev/sdj1       3.0T   77M  2.9T   1% /hadoop/data07
/dev/sdk1       3.0T   77M  2.9T   1% /hadoop/data08
/dev/sdl1       3.0T   77M  2.9T   1% /hadoop/data09
/dev/sdm1       3.0T   77M  2.9T   1% /hadoop/data10



# Restore the DataNode

Check the health of whole HDFS...

In [47]:
!ansible hadoop_client -s -U hdfs -a 'hdfs dfsadmin -report' -l {target_group}

XXX.XXX.XXX.200 | SUCCESS | rc=0 >>
Configured Capacity: 238194899124224 (216.64 TB)
Present Capacity: 226085043405016 (205.62 TB)
DFS Remaining: 222783090728298 (202.62 TB)
DFS Used: 3301952676718 (3.00 TB)
DFS Used%: 1.46%
Under replicated blocks: 0
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 746

-------------------------------------------------
Live datanodes (8):

Name: XXX.XXX.XXX.226:1004 (sn02022001)
Hostname: sn02022001
Decommission Status : Normal
Configured Capacity: 29528238325760 (26.86 TB)
DFS Used: 518379108592 (482.78 GB)
Non DFS Used: 1501183849997 (1.37 TB)
DFS Remaining: 27508675367171 (25.02 TB)
DFS Used%: 1.76%
DFS Remaining%: 93.16%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 8
Last contact: Fri Sep 02 19:19:31 JST 2016


Name: XXX.XXX.XXX.232:1004 (sn02032001)
Hostname: sn02032001
Decommission Status : Normal
Configured Capa

## Setting up the DataNode

Let's prepare the DataNode directories.

Ensure that the directory **/hadoop/dataXX/dfs/datadir** is present...

In [48]:
import os
import tempfile

work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmpP0U4b9'

In [50]:
!rm -fr {work_dir}/hadoop
!git clone https://github.com/NII-cloud-operation/Literate-computing-Hadoop.git {work_dir}/hadoop
!tree {work_dir}/hadoop

Cloning into '/tmp/tmpP0U4b9/hadoop'...
remote: Counting objects: 849, done.
remote: Total 849 (delta 0), reused 0 (delta 0), pack-reused 849
Receiving objects: 100% (849/849), 169.06 KiB | 267.00 KiB/s, done.
Resolving deltas: 100% (272/272), done.
Checking connectivity... done.
/tmp/tmpP0U4b9/hadoop
└── playbooks
    ├── conf_base.retry
    ├── conf_base.yml
    ├── conf_hdfs_base.yml
    ├── conf_hdfs_spark.yml
    ├── conf_hdfs_tez.yml
    ├── conf_hdfs_yarn.yml
    ├── conf_namenode_bootstrapstandby.yml
    ├── conf_tez.yml
    ├── enter_hdfs_safemode.yml
    ├── format_namenode.yml
    ├── group_vars
    │   └── all
    │       ├── base
    │       ├── cgroups
    │       ├── collect
    │       ├── f500.dumpall
    │       ├── hbase_master
    │       ├── hbase_regionserver
    │       ├── hcatalog
    │       ├── hdfs_base
    │       ├── hdfs_spark
    │       ├── hdfs_tez
    │       ├── hdfs_yarn
    │       ├── hive
    │       ├── httpfs
    │       ├── hue
    │       ├──

In [51]:
playbook_dir = os.path.join(work_dir, 'hadoop/playbooks')
!ls -la {playbook_dir} | head

total 244
drwxr-xr-x  4 root root 4096 Sep  2 19:20 .
drwxr-xr-x  4 root root 4096 Sep  2 19:20 ..
-rw-r--r--  1 root root   13 Sep  2 19:20 conf_base.retry
-rw-r--r--  1 root root   39 Sep  2 19:20 conf_base.yml
-rw-r--r--  1 root root  136 Sep  2 19:20 conf_hdfs_base.yml
-rw-r--r--  1 root root  137 Sep  2 19:20 conf_hdfs_spark.yml
-rw-r--r--  1 root root  135 Sep  2 19:20 conf_hdfs_tez.yml
-rw-r--r--  1 root root  136 Sep  2 19:20 conf_hdfs_yarn.yml
-rw-r--r--  1 root root  188 Sep  2 19:20 conf_namenode_bootstrapstandby.yml


In [52]:
!ansible-playbook -CDv {playbook_dir}/install_slavenode.yml -l { host_new_machine }

Using /etc/ansible/ansible.cfg as config file

PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [base : include] **********************************************************
included: /tmp/tmpP0U4b9/hadoop/playbooks/roles/base/tasks/repo.yml for XXX.XXX.XXX.233

TASK [base : install_hdp_repo] *************************************************
ok: [XXX.XXX.XXX.233] => {"changed": false, "gid": 0, "group": "root", "mode": "0644", "owner": "root", "path": "/etc/yum.repos.d/hdp.repo", "size": 556, "state": "file", "uid": 0}

TASK [base : include] **********************************************************
included: /tmp/tmpP0U4b9/hadoop/playbooks/roles/base/tasks/conf.yml for XXX.XXX.XXX.233

TASK [base : create_hadoop_conf_dir] *******************************************
ok: [XXX.XXX.XXX.233] => {"changed": false, "gid": 0, "group": "root", "mode": "075

In [53]:
!ansible-playbook {playbook_dir}/install_slavenode.yml -l { host_new_machine }


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [base : include] **********************************************************
included: /tmp/tmpP0U4b9/hadoop/playbooks/roles/base/tasks/repo.yml for XXX.XXX.XXX.233

TASK [base : install_hdp_repo] *************************************************
ok: [XXX.XXX.XXX.233]

TASK [base : include] **********************************************************
included: /tmp/tmpP0U4b9/hadoop/playbooks/roles/base/tasks/conf.yml for XXX.XXX.XXX.233

TASK [base : create_hadoop_conf_dir] *******************************************
ok: [XXX.XXX.XXX.233]

TASK [base : copy_conf_files] **************************************************
ok: [XXX.XXX.XXX.233] => (item=core-site.xml)
ok: [XXX.XXX.XXX.233] => (item=hdfs-site.xml)
ok: [XXX.XXX.XXX.233] => (item=yarn-site.xml)
ok: [XXX.XXX.XXX.233] => (item=mapred-site.xml

## Remove the node from the decommission list

Remove the DataNode from decommission list...

In [54]:
!cp group_vars/hadoop_all_cluster1 {work_dir}/hadoop_all_cluster1_old

Edit [hadoop_all_cluster1](../edit/group_vars/hadoop_all_cluster1) and remove it from `datanode_decommission_nodes`

It has not been decommissioned, skipped.

In [55]:
#!diff -ur {work_dir}/hadoop_all_cluster1_old group_vars/hadoop_all_cluster1

Deliver latest configurations to all nodes...

In [56]:
#!ansible-playbook -CDv {playbook_dir}/conf_base.yml -l {target_group}

In [57]:
#!ansible-playbook {playbook_dir}/conf_base.yml -l {target_group}

OK, I will ask NameNodes to reload node settings...

In [58]:
#!ansible hadoop_namenode -m shell -a 'hdfs dfsadmin -fs hdfs://$(hostname):8020 -refreshNodes' --sudo --sudo-user hdfs -l {target_group}

## cgroups

Start cgroups...

In [59]:
!ansible-playbook {playbook_dir}/install-base.yml -l { host_new_machine }


PLAY [hadoop_all] **************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [os : include] ************************************************************
included: /tmp/tmpP0U4b9/hadoop/playbooks/roles/os/tasks/limits.yml for XXX.XXX.XXX.233

TASK [os : set_nofile_soft_limit] **********************************************
ok: [XXX.XXX.XXX.233]

TASK [os : set_nofile_hard_limit] **********************************************
ok: [XXX.XXX.XXX.233]

TASK [os : set_core_soft_limit] ************************************************
ok: [XXX.XXX.XXX.233]

TASK [os : set_core_hard_limit] ************************************************
ok: [XXX.XXX.XXX.233]

TASK [os : include] ************************************************************
included: /tmp/tmpP0U4b9/hadoop/playbooks/roles/os/tasks/thp.yml for XXX.XXX.XXX.233

TASK [os : set_transparent_hugepage] ***********************

## Start the DataNode

Start the DataNode...

In [60]:
!ansible-playbook {playbook_dir}/start_datanode.yml -l { host_new_machine }


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [start_hadoop-hdfs-datanode] **********************************************
changed: [XXX.XXX.XXX.233]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.233              : ok=2    changed=1    unreachable=0    failed=0   



OK. Checking the health...

In [61]:
!ansible hadoop_client -s -U hdfs -a 'hdfs dfsadmin -report' -l {target_group}

XXX.XXX.XXX.200 | SUCCESS | rc=0 >>
Configured Capacity: 268707633709056 (244.39 TB)
Present Capacity: 255047025981280 (231.96 TB)
DFS Remaining: 251740821216440 (228.96 TB)
DFS Used: 3306204764840 (3.01 TB)
DFS Used%: 1.30%
Under replicated blocks: 0
Blocks with corrupt replicas: 0
Missing blocks: 0
Missing blocks (with replication factor 1): 746

-------------------------------------------------
Live datanodes (9):

Name: XXX.XXX.XXX.226:1004 (sn02022001)
Hostname: sn02022001
Decommission Status : Normal
Configured Capacity: 29528238325760 (26.86 TB)
DFS Used: 516681058899 (481.20 GB)
Non DFS Used: 1501183657196 (1.37 TB)
DFS Remaining: 27510373609665 (25.02 TB)
DFS Used%: 1.75%
DFS Remaining%: 93.17%
Configured Cache Capacity: 0 (0 B)
Cache Used: 0 (0 B)
Cache Remaining: 0 (0 B)
Cache Used%: 100.00%
Cache Remaining%: 0.00%
Xceivers: 8
Last contact: Fri Sep 02 19:24:04 JST 2016


Name: XXX.XXX.XXX.232:1004 (sn02032001)
Hostname: sn02032001
Decommission Status : Normal
Configured Capa

OK, It seems that the cluster is HELATHY.

> Though `Missing blocks (with replication factor 1)` of the sample output shows `746`, it is caused by [known issue HDFS-8806](https://issues.apache.org/jira/browse/HDFS-8806).
> We can ignore the counter.

Additionaly, I would like to check the result of fsck...

In [62]:
!ansible hadoop_client -s -U hdfs -a 'hdfs fsck /' -l {target_group}

XXX.XXX.XXX.200 | SUCCESS | rc=0 >>
FSCK started by hdfs (auth:KERBEROS_SSL) from /XXX.XXX.XXX.200 for path / at Fri Sep 02 19:24:30 JST 2016
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
..................................................

## Start the NodeManager

In [63]:
!ansible-playbook {playbook_dir}/start_nodemanager.yml -l { host_new_machine }


PLAY [hadoop_slavenode] ********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [start_hadoop-yarn-nodemanager] *******************************************
changed: [XXX.XXX.XXX.233]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.233              : ok=2    changed=1    unreachable=0    failed=0   



Confirm the YARN nodes...

In [64]:
!ansible hadoop_client -s -U yarn -a 'yarn node -list' -l {target_group}

XXX.XXX.XXX.200 | SUCCESS | rc=0 >>
Total Nodes:9
         Node-Id	     Node-State	Node-Http-Address	Number-of-Running-Containers
sn02030401:45454	        RUNNING	  sn02030401:8044	                           0
sn02032001:45454	        RUNNING	  sn02032001:8044	                           0
sn02021201:45454	        RUNNING	  sn02021201:8044	                           0
sn02022001:45454	        RUNNING	  sn02022001:8044	                           0
sn02022401:45454	        RUNNING	  sn02022401:8044	                           0
sn02031601:45454	        RUNNING	  sn02031601:8044	                           0
sn02020401:45454	        RUNNING	  sn02020401:8044	                           0
sn02031201:45454	        RUNNING	  sn02031201:8044	                           0
sn02032401:45454	        RUNNING	  sn02032401:8044	                           016/09/02 19:24:52 INFO impl.TimelineClientImpl: Timeline service address: https://cn01070403:8190/ws/v1/timeline/



**sn02031601** is running, OK.

## Start the RegionServer

In [65]:
!ansible-playbook {playbook_dir}/start_hbase_regionserver.yml -l { host_new_machine }


PLAY [hadoop_hbase_regionserver] ***********************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.233]

TASK [start_hbase-regionserver] ************************************************
changed: [XXX.XXX.XXX.233]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.233              : ok=2    changed=1    unreachable=0    failed=0   



Confirm the Region Servers...

In [66]:
from kazoo.client import KazooClient
zk_stdout = !ansible -l {target_group} -m ping hadoop_zookeeperserver
zk_hosts = [line.split()[0] for line in zk_stdout if "SUCCESS" in line]
zk = KazooClient(hosts='%s:2181' % zk_hosts[0], read_only=True)
zk.start()
(master_result,v) = zk.get("/hbase/master")
zk.stop()
active_master = None
for m in filter(lambda m: m['HBase Master'], machines):
    if m['Name'] in master_result:
        active_master = m['Service IP']
active_master

'XXX.XXX.XXX.197'

In [67]:
!zk-shell { zk_hosts[0] } --run-once "ls /hbase/rs"

sn02020401,16020,1468567893041
sn02021201,16020,1468567892900
sn02022001,16020,1468567893194
sn02022401,16020,1468567894744
sn02030401,16020,1470634628613
sn02031201,16020,1458102803012
sn02031601,16020,1472811915179
sn02032001,16020,1469169835467
sn02032401,16020,1470720586071


**sn02031601** is included in the list of Region Servers, OK.

It's done.

# Cleanup

Remove temporary files...

In [68]:
!rm -fr {work_dir}